In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
from torch import tensor
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=4,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=4,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=4,
                          num_workers=4,
                          shuffle=True)

In [3]:
print(train_dataset[0][0].size())
print(train_dataset[0][1].size())
print(train_dataset[0][2].size())
print(len(train_dataset[0]))
print(train_dataset[0][1].sum())

torch.Size([3, 1024, 2048])
torch.Size([34, 1024, 2048])
torch.Size([1024, 2048])
3
tensor(2097152.)


In [4]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)
        
epochs     = 100
criterion = torch.nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    fcn_model.train(True)
    
    for epoch in range(epochs):
        ts = time.time()
        for train_idx, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.to('cuda')# Move your inputs onto the gpu
                labels = Y.to('cuda') # Move your labels onto the gpu
            else:
                inputs, labels = X, Y # Unpack variables into inputs and labels
                
#             print('inputs.device', inputs.device)    

            outputs = fcn_model(inputs)
#             print('outputs size: ', outputs.size())
#             print('labels size: ', labels.size())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if train_idx % 10 == 0:
                print("epoch{}, iter{} out of {}, loss: {}".format(epoch, train_idx, len(train_loader), loss.item()))
                
                
            # Checking if early stop conditions met
#             if early_stop and i > early_stop_epoch:
#                 shouldStop = True
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val_loss, val_acc, IOU = val(epoch)


def val(epoch):
    
    # set to evaluation mode 
#     fcn_model.eval()
    fcn_model.train(False)

    softmax = nn.Softmax(dim = 1)
    
    loss = []
    pred = []
    acc = []
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(val_loader):
        optimizer.zero_grad()
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
            
        if use_gpu:
            inputs = X.to('cuda')
            labels = Y.to('cuda')
            tar = tar.to('cuda')
        else:
            inputs, labels, tar = X, Y, tar 

#         print('inputs.device', inputs.device)    
        
        outputs = fcn_model(inputs)
        print(outputs.size())
        print(labels.size())
#         loss.append(criterion(outputs, labels.long()))
        loss.append(criterion(outputs, labels).item())
        prediction = softmax(outputs)
#         temp = prediction.data.cpu()
#         print(type(temp))
#         print(temp.size())
        acc.append(pixel_acc(prediction.data.cpu(), labels.data.cpu()))
#         IOU = IOU + np.array(iou(pred, Y))
        IOU = IOU + np.array(iou(prediction.data.cpu(), labels.data.cpu()))
    
    acc = sum(acc)/len(acc)
    avg_loss = sum(loss)/len(loss) 
    IOU = IOU/iter  
    
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    
    return avg_loss, acc, IOU      
       
    
    
    
def test():
    
    softmax = nn.Softmax(dim = 1)
    
    pred = []
    acc = []
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(test_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
        
        if use_gpu:
            inputs = X.to('cuda')
            labels = Y.to('cuda')
            tar = tar.to('cuda')
        else:
            inputs, labels, tar = X, Y, tar 
                
        print('inputs.device', inputs.device)    

        outputs = fcn_model(inputs)  
        print(output.shape)
        prediction = softmax(outputs)
        acc.append(pixel_acc(prediction.cpu(), labels.cpu()))
        IOU = IOU + np.array(iou(prediction, Y))
        
    acc = sum(acc)/len(acc)        
    IOU = IOU/iter

    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
    return acc, IOU
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    train()

xshape:  torch.Size([4, 3, 1024, 2048])
in encoder:  torch.Size([4, 32, 512, 1024])
in encoder:  torch.Size([4, 64, 256, 512])
in encoder:  torch.Size([4, 128, 128, 256])
in encoder:  torch.Size([4, 256, 64, 128])
in encoder:  torch.Size([4, 512, 32, 64])
out_encoder size:  torch.Size([4, 512, 32, 64])
in decoder:  torch.Size([4, 512, 64, 128])
in decoder:  torch.Size([4, 256, 128, 256])
in decoder:  torch.Size([4, 128, 256, 512])
in decoder:  torch.Size([4, 64, 512, 1024])
in decoder:  torch.Size([4, 32, 1024, 2048])
out_decoder size:  torch.Size([4, 32, 1024, 2048])
score size:  torch.Size([4, 34, 1024, 2048])
torch.Size([4, 34, 1024, 2048])
torch.Size([4, 1024, 2048])
xshape:  torch.Size([4, 3, 1024, 2048])
in encoder:  torch.Size([4, 32, 512, 1024])
in encoder:  torch.Size([4, 64, 256, 512])
in encoder:  torch.Size([4, 128, 128, 256])
in encoder:  torch.Size([4, 256, 64, 128])
in encoder:  torch.Size([4, 512, 32, 64])
out_encoder size:  torch.Size([4, 512, 32, 64])
in decoder:  tor

RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 10.92 GiB total capacity; 10.40 GiB already allocated; 3.50 MiB free; 4.11 MiB cached)